In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
df = pd.read_csv('Resources/filled_final_table.csv')
df.head()

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks
0,2022-06,2022,Alamance,382500,298,18,452,12,108,2112,443362,644,NaN,5.52,NaN,0.034063,54.0
1,2022-06,2022,Alexander,312450,34,35,28,0,20,1942,370648,59,NaN,5.52,NaN,0.016428,25.0
2,2022-06,2022,Alleghany,389000,54,39,24,0,20,1904,422802,92,NaN,5.52,NaN,-0.038200,0.0
3,2022-06,2022,Anson,170000,22,34,16,0,8,1502,262240,48,NaN,5.52,NaN,0.218638,3.0
4,2022-06,2022,Ashe,485000,76,31,48,0,20,1907,592879,137,NaN,5.52,NaN,-0.088346,5.0


In [4]:
df.dtypes

Date                        object
Year                         int64
County                      object
median_listing_price         int64
active_listing_count         int64
median_days_on_market        int64
new_listing_count            int64
price_increased_count        int64
price_reduced_count          int64
median_square_feet           int64
average_listing_price        int64
total_listing_count          int64
violent_crime_rate          object
30_Year_Fixed_Rate         float64
Number_of_Schools          float64
median_listing_price_mm    float64
Number_of_Parks            float64
dtype: object

In [5]:

df['violent_crime_rate']= df['violent_crime_rate'].astype(str).str.replace(',','').astype(float)
df.dtypes

Date                        object
Year                         int64
County                      object
median_listing_price         int64
active_listing_count         int64
median_days_on_market        int64
new_listing_count            int64
price_increased_count        int64
price_reduced_count          int64
median_square_feet           int64
average_listing_price        int64
total_listing_count          int64
violent_crime_rate         float64
30_Year_Fixed_Rate         float64
Number_of_Schools          float64
median_listing_price_mm    float64
Number_of_Parks            float64
dtype: object

In [6]:
df["Year"].unique()

array([2022, 2021, 2020, 2019, 2018, 2017, 2016])

In [7]:
filter_df = df[df['Year'] <2019]

In [8]:
filter_df['Price_Status'] = np.where(filter_df['median_listing_price_mm']>=0, 1, 0)

In [9]:
filter_df.head()

,Date,Year,County,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,median_square_feet,average_listing_price,total_listing_count,violent_crime_rate,30_Year_Fixed_Rate,Number_of_Schools,median_listing_price_mm,Number_of_Parks,Price_Status
4196,2018-12,2018,Alamance,249900,761,78,172,28,116,2060,276731,843,419.9,4.64,94.0,0.021635,54.0,1
4197,2018-12,2018,Alexander,229900,90,102,8,0,16,1924,258096,103,221.9,4.64,49.0,0.108219,25.0,1
4198,2018-12,2018,Alleghany,114900,420,227,4,0,20,1886,179920,438,114.8,4.64,11.0,-0.000870,0.0,0
4199,2018-12,2018,Anson,125000,65,100,8,0,12,1500,152823,78,568.4,4.64,15.0,-0.037721,3.0,0
4200,2018-12,2018,Ashe,65000,899,214,28,0,52,1776,157405,957,NaN,4.64,7.0,0.000770,5.0,1


In [10]:
columns = [
    "Date", "Year", "County",
    "active_listing_count", "median_days_on_market", "new_listing_count",
    "price_increased_count", "price_reduced_count", "median_square_feet",
    "total_listing_count", "violent_crime_rate","Number_of_Parks",
    "Number_of_Schools","30_Year_Fixed_Rate", 
     "Price_Status"]

target = ["Price_Status"]

In [11]:
# Load the data

df_data = filter_df.loc[:, columns].copy()
df_data = df_data.dropna()

df_data.reset_index(inplace=True, drop=True)

df_data.head()
df_data.isnull().sum()

Date                     0
Year                     0
County                   0
active_listing_count     0
median_days_on_market    0
new_listing_count        0
price_increased_count    0
price_reduced_count      0
median_square_feet       0
total_listing_count      0
violent_crime_rate       0
Number_of_Parks          0
Number_of_Schools        0
30_Year_Fixed_Rate       0
Price_Status             0
dtype: int64

In [12]:
# Create our features
X = df_data.drop('Price_Status', axis=1)

X = pd.get_dummies(X)

# Create our target
y = df_data.loc[:, target].copy()

In [13]:
# Check the balance of our target values
y['Price_Status'].value_counts()

1    1448
0    1162
Name: Price_Status, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'Price_Status': 1})

In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(X.dtypes)

Year                       int64
active_listing_count       int64
median_days_on_market      int64
new_listing_count          int64
price_increased_count      int64
price_reduced_count        int64
median_square_feet         int64
total_listing_count        int64
violent_crime_rate       float64
Number_of_Parks          float64
Number_of_Schools        float64
30_Year_Fixed_Rate       float64
Date_2016-07               uint8
Date_2016-08               uint8
Date_2016-09               uint8
Date_2016-10               uint8
Date_2016-11               uint8
Date_2016-12               uint8
Date_2017-01               uint8
Date_2017-02               uint8
Date_2017-03               uint8
Date_2017-04               uint8
Date_2017-05               uint8
Date_2017-06               uint8
Date_2017-07               uint8
Date_2017-08               uint8
Date_2017-09               uint8
Date_2017-10               uint8
Date_2017-11               uint8
Date_2017-12               uint8
Date_2018-

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'Price_Status': 1})

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.49836468475281215

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[156, 142],
       [187, 168]])

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.45      0.52      0.47      0.49      0.50      0.25       298
          1       0.54      0.47      0.52      0.51      0.50      0.25       355

avg / total       0.50      0.50      0.50      0.50      0.50      0.25       653



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Price_Status': 1})

In [22]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5017203894507988

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[158, 140],
       [187, 168]])

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.46      0.53      0.47      0.49      0.50      0.25       298
          1       0.55      0.47      0.53      0.51      0.50      0.25       355

avg / total       0.51      0.50      0.50      0.50      0.50      0.25       653



### Undersampling

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

### Combination (Over and Under) Sampling


In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs')
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [1]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

NameError: name 'confusion_matrix' is not defined

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))